In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
from os import path
from zipfile import ZipFile
import pydub
from pydub import AudioSegment
import librosa
import soundfile as sf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,recall_score,accuracy_score,f1_score,precision_score
import pickle

In [7]:
#Setting Main Path
MainPath = "/home/saleh/Documents/NUCES/Sem 3/PAI/i20-0605"
os.chdir(MainPath)

In [8]:
#Find te zip folder and extract
File = os.listdir()
chosen = ""
for x in File:
    if(x.find(".zip") > 0):
        chosen = x
        break

with ZipFile(chosen, 'r') as zipObj:
    zipObj.extractall()

In [9]:
#Initializing Arrays
NPListLabel = np.empty(1)
ListMFCC = [[0]]
ListMFCCTranspose = [[0]]
ListChroma = [[0]]

In [10]:
#Reconvert the audios to wav format
os.chdir(MainPath + "/ProjectAllfiles")
ListRecordingFolders = sorted(os.listdir())
for Labels in ListRecordingFolders:
    TempPath = MainPath + "/ProjectAllfiles/" + Labels
    os.chdir(TempPath)
    for Recording in os.listdir():
        InputFile = Recording
        OutPutFile = InputFile.replace("mp3","wav").replace("m4a","wav").replace("aac","wav").replace("mpeg","wav")
        sound = AudioSegment.from_file(InputFile)
        sound.export(OutPutFile, format = "wav")
    os.chdir("../")

In [11]:
#Finding features from the audios
os.chdir(MainPath + "/ProjectAllfiles")
for Folder in sorted(os.listdir()):
    TempPath = MainPath + "/ProjectAllfiles"+ "/" + Folder
    os.chdir(TempPath)
    for recordingFile in os.listdir():
        NPListLabel = np.append(NPListLabel,Folder)
        
        X , SR = librosa.load(recordingFile,res_type='kaiser_fast')
        
        #Trimming the scilent parts of the audio from the ends
        Trim, index = librosa.effects.trim(X, top_db = 10)
        
        #MFCC Features
        MFCC = librosa.feature.mfcc(y = Trim, sr = SR)
        ListMFCC.append(MFCC.flatten())
        
        #MFCC Transpose Features
        MFCC_T = (MFCC.T)
        ListMFCCTranspose.append(MFCC_T.flatten())
        
        #STFT features
        C = np.abs(librosa.stft(Trim))
        chroma = librosa.feature.chroma_stft(y = C.flatten(), sr = SR)
        ListChroma.append(chroma.flatten())

    os.chdir("../")

In [12]:
#Function to sort a list
def sortList(ListLengths):
    for x in range(0,len(ListLengths)):
        for y in range(0,len(ListLengths)-1):
            if(ListLengths[y][0] > ListLengths[y+1][0]):
                ListLengths[y],ListLengths[y+1] = ListLengths[y+1],ListLengths[y]
    return ListLengths

In [13]:
DF = pd.DataFrame()

ListMFCC = ListMFCC[1:]
ListChroma = ListChroma[1:]
NPListLabel = NPListLabel[1:]
ListMFCCTranspose = ListMFCCTranspose[1:]

ListLengths = []
count = -1
for x in ListMFCC:
    count += 1
    ListLengths.append([len(x),count,x])

ListLengths = sortList(ListLengths)
    
ListDeleted = []
ListDeleted.append(ListLengths[0])
ListDeleted.append(ListLengths[len(ListLengths)-1])

ListLengths = ListLengths[1:len(ListLengths)-1]

#Label encoding labels column
le = LabelEncoder()
NEWListLabel = le.fit_transform(NPListLabel)

DF["Label"] = NEWListLabel

#Finding max length of MFCC features
maxLengthMFCC = 0
for x in ListLengths:
    if(x[0] > maxLengthMFCC):
        maxLengthMFCC = x[0]

#Finding max length of chroma features
maxLengthChroma = 0
for x in ListChroma:
    if(len(x) > maxLengthChroma):
        maxLengthChroma = len(x)

#Finding max length of Transpose of MFCC features
maxLengthMFCCTranspose = 0
for x in ListMFCCTranspose:
    if(len(x) > maxLengthMFCCTranspose):
        maxLengthMFCCTranspose = len(x)
        
maxLength = (max(maxLengthMFCC, maxLengthChroma, maxLengthMFCCTranspose))

#Padding all three feature lists
NewMFCCList = []
for x in ListMFCC:
    if(len(x) <= maxLength):
        NewMFCCList.append(np.pad(x,(0,maxLength-len(x))))
    else:
        NewMFCCList.append(x)

NewChromaList = []
for x in ListChroma:
    if(len(x) <= maxLength):
        NewChromaList.append(np.pad(x,(0,maxLength-len(x))))
    else:
        NewChromaList.append(x)

NewMFCCTransposeList = []
for x in ListMFCCTranspose:
    if(len(x) <= maxLength):
        NewMFCCTransposeList.append(np.pad(x,(0,maxLength-len(x))))
    else:
        NewMFCCTransposeList.append(x)

#Concatenating padded lists into 1
New = []
for x in range(0,len(NewMFCCList)):
    temp = np.concatenate((NewMFCCList[x],NewChromaList[x],NewMFCCTransposeList[x]))
    New.append(temp)

#Column of Features
DF["Features"] = New 

#Deleting (3) rows which had outlier MFCC features 
for x in ListDeleted:
    DF = DF.drop(x[1])

In [14]:
#Data Frame
DF = DF.set_index(np.arange(len(DF)))
print(DF)

      Label                                           Features
0         0  [-437.9129, -428.386, -421.04306, -425.20978, ...
1         0  [-299.16992, -287.1158, -298.21167, -314.1415,...
2         0  [-346.73273, -281.279, -248.5343, -240.8198, -...
3         0  [-532.15173, -513.82043, -498.40793, -488.2128...
4         0  [-325.96017, -329.19733, -336.58618, -339.5479...
...     ...                                                ...
5623      3  [-237.78828, -238.48456, -265.4574, -299.4738,...
5624      3  [-360.7449, -343.01395, -341.77145, -349.5287,...
5625      3  [-307.32532, -304.06332, -306.60562, -295.0947...
5626      3  [-309.28964, -268.9655, -266.18445, -276.3862,...
5627      3  [-307.11652, -308.4651, -309.37866, -291.21887...

[5628 rows x 2 columns]


In [15]:
os.chdir(MainPath)

In [16]:
#Get training and testing data
X_train, X_test, y_train, y_test = train_test_split(DF["Features"], DF["Label"], test_size = 0.2,random_state = 40)

In [17]:
#Converting to lists to avoid any error while training
x_train = list(X_train)
x_test = list(X_test)

In [18]:
#Applying Standard Sclarer
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)
x_train_sc = sc.transform(x_train)
x_test_sc = sc.transform(x_test)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
ModelKNeighbours = KNeighborsClassifier(n_neighbors = 3,metric= "braycurtis", p = 1 )
ModelKNeighbours.fit(x_train_sc, y_train)
pred = ModelKNeighbours.predict(x_test_sc)

In [20]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred,average = 'micro')*100)
print("Precision :", precision_score(y_test, pred,average = 'micro')*100)
print("F1 Score :",f1_score(y_test, pred, average = 'micro')*100)

[[ 60  17   9  17]
 [ 27 551  13  37]
 [ 45  47  19   7]
 [ 35 116   9 117]]
Accuracy : 66.34103019538188
Recall : 66.34103019538188
Precision : 66.34103019538188
F1 Score : 66.34103019538188


In [21]:
FileName = 'i200605_KNN.pkl'
pickle.dump(ModelKNeighbours, open(FileName,'wb'))

In [22]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy', splitter = "best")
clf = clf.fit(x_train_sc, y_train)
pred = clf.predict(x_test_sc)

In [23]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred,average = 'micro')*100)
print("Precision :", precision_score(y_test, pred,average = 'micro')*100)
print("F1 Score :",f1_score(y_test, pred, average='micro')*100)

[[ 40  23  13  27]
 [ 19 500  45  64]
 [ 18  52  35  13]
 [ 14  54  20 189]]
Accuracy : 67.85079928952042
Recall : 67.85079928952042
Precision : 67.85079928952042
F1 Score : 67.85079928952042


In [24]:
FileName = 'i200605_DecisionTree.pkl'
pickle.dump(clf, open(FileName,'wb'))

In [25]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter = 500)
mlp.fit(x_train_sc , y_train)
pred = mlp.predict(x_test_sc)

In [26]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='macro')*100)
print("F1 Score :",f1_score(y_test, pred, average='macro')*100)

[[ 72  12   8  11]
 [  4 584  13  27]
 [  4  37  68   9]
 [  2  42   1 232]]
Accuracy : 84.90230905861456
Recall : 76.06954361851362
Precision : 83.25826849666346
F1 Score : 79.07874614239837


In [27]:
FileName = 'i200605_MLP.pkl'
pickle.dump(mlp, open(FileName,'wb'))

In [28]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(x_train_sc , y_train)
pred = RFC.predict(x_test_sc)

In [29]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='macro')*100)
print("F1 Score :",f1_score(y_test, pred, average='macro')*100)

[[ 44  47   1  11]
 [  0 621   2   5]
 [  6  90  21   1]
 [  6  91   0 180]]
Accuracy : 76.90941385435168
Recall : 56.09558913709709
Precision : 82.64671581803785
F1 Score : 61.24052854308001


In [30]:
FileName = 'i200605_RandomForestClassifier.pkl'
pickle.dump(RFC, open(FileName,'wb'))

In [31]:
from sklearn.ensemble import ExtraTreesClassifier
ETC = ExtraTreesClassifier()
ETC.fit(x_train_sc , y_train)
pred = ETC.predict(x_test_sc)

In [32]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='macro')*100)
print("F1 Score :",f1_score(y_test, pred, average='macro')*100)

[[ 39  60   0   4]
 [  2 616   3   7]
 [  6  97  13   2]
 [  5 146   1 125]]
Accuracy : 70.42628774422735
Recall : 48.024138146895325
Precision : 77.26991953520775
F1 Score : 52.365203202404


In [33]:
FileName = 'i200605_ExtraTreesClassifier.pkl'
pickle.dump(ETC, open(FileName,'wb'))

In [34]:
from sklearn.ensemble import BaggingClassifier
GBC = BaggingClassifier()
GBC.fit(x_train_sc , y_train)
pred = GBC.predict(x_test_sc)

In [35]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='macro')*100)
print("F1 Score :",f1_score(y_test, pred, average='macro')*100)

[[ 41  39   5  18]
 [  6 594   8  20]
 [  9  79  25   5]
 [ 13  66   3 195]]
Accuracy : 75.93250444049734
Recall : 56.49384127379711
Precision : 69.66957177907516
F1 Score : 59.83603404052168


In [36]:
FileName = 'i200605_BaggingClassifier.pkl'
pickle.dump(GBC, open(FileName,'wb'))

In [37]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
ORC = OneVsRestClassifier(SVC(), n_jobs = 4)
ORC.fit(x_train_sc , y_train)
pred = ORC.predict(x_test_sc)

In [38]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='micro')*100)
print("F1 Score :",f1_score(y_test, pred, average='micro')*100)

[[ 55  38   0  10]
 [  1 617   1   9]
 [  7  88  17   6]
 [  5  98   0 174]]
Accuracy : 76.64298401420959
Recall : 57.21728250832263
Precision : 76.64298401420959
F1 Score : 76.6429840142096


In [39]:
FileName = 'i200605_OneVsRestClassifier.pkl'
pickle.dump(ORC, open(FileName,'wb'))

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(n_estimators = 15,learning_rate = .3)
GBC.fit(x_train_sc , y_train)
pred = GBC.predict(x_test_sc)

In [41]:
print(confusion_matrix(y_test,pred))
print("Accuracy :" , accuracy_score(y_test,pred)*100)  
print("Recall :", recall_score(y_test, pred, average='macro')*100)
print("Precision :", precision_score(y_test, pred, average='macro')*100)
print("F1 Score :",f1_score(y_test, pred, average='macro')*100)

[[ 42  40   7  14]
 [ 10 595   6  17]
 [ 10  77  22   9]
 [  7  64   6 200]]
Accuracy : 76.28774422735346
Recall : 56.59203895516366
Precision : 68.63417321701232
F1 Score : 59.659359776642454


In [42]:
FileName = 'i200605_GradientBoostingClassifier.pkl'
pickle.dump(GBC, open(FileName,'wb'))